<a href="https://colab.research.google.com/github/566aass/colab/blob/colab_code/sample_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CNNN

import ee
import geemap
import json
import time
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dropout,Dense
from sklearn.model_selection import train_test_split  # 导入 train_test_split 函数
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix
admin_boundaries = ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM1")
# 筛选山东省的边界
shandong_boundary = admin_boundaries.filter(ee.Filter.stringStartsWith('shapeName', 'Shandong')).first()
sentinel2_sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
start_date = '2023-03-01'
end_date = '2023-06-01'
# 过滤影像集
filtered_sentinel2 = sentinel2_sr.filterBounds(shandong_boundary.geometry()) \
                                 .filterDate(start_date, end_date)

def cloudShadowMask(img):
    qa = img.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    cloudMask = qa.bitwiseAnd(cloudBitMask).eq(0)
    cirrusMask = qa.bitwiseAnd(cirrusBitMask).eq(0)
    mask = cloudMask.And(cirrusMask)
    kernel = ee.Kernel.circle(radius=5, units='pixels')  # 定义一个圆形卷积核
    mean_image = img.reduceNeighborhood(ee.Reducer.mean(), kernel).updateMask(mask)  # 计算平均值，并应用掩膜
    # 使用平均值填充云像素
    filled_image = img.updateMask(mask).unmask(mean_image)

    return filled_image

filtered_sentinel2 = filtered_sentinel2.map(cloudShadowMask)

mosaicked_image = filtered_sentinel2.mosaic()
mosaicked_image = mosaicked_image.clip(shandong_boundary.geometry())

excel_file = '/content/drive/MyDrive/train/merged_train_data.xlsx'
train_df = pd.read_excel(excel_file)

X_train = train_df.drop(columns=['class'])
y_train = train_df['class']

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
# 构建CNN
model = tf.keras.models.Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(y_train.shape[1], activation='softmax')
])
# 编译
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)